In [9]:
import pandas as pd
import numpy as np
import os
import requests
import re
import funciones as mf

In [10]:
data = pd.read_csv('biblioteca_popular.csv', encoding='utf8')
data_biblio = pd.DataFrame(data)
data = pd.read_csv('cine.csv', encoding='utf8')
data_cine = pd.DataFrame(data)
data = pd.read_csv('museos_datosabiertos.csv', encoding='utf8')
data_museo = pd.DataFrame(data)

biblio_c = data_biblio.copy()
cine_c = data_cine.copy()
museo_c = data_museo.copy()

<h1>NORMALIZANDO TABLA BIBLIOTECA</h1>

In [11]:
#Renombrando columnas
biblio_c.rename(columns={'Cod_Loc': 'cod_localidad', 'IdProvincia': 'id_provincia','IdDepartamento': 'id_departamento','Categoría': 'categoria','Provincia': 'provincia','Localidad': 'localidad','Nombre': 'nombre','Domicilio': 'domicilio','CP': 'codigo postal','Teléfono': 'numero de telefono','Mail': 'mail','Web': 'web', 'Fuente': 'fuente'}, inplace=True)

In [12]:
#Reemplazo los datos vacios por s/d
biblio_c.fillna('s/d', inplace=True)

In [13]:
#Viendo si hay claves duplicadas en cod_localidad
biblio_c[['cod_localidad','localidad']].groupby(['cod_localidad','localidad']).count().head(20)

,
cod_localidad,localidad
2000010,Ciudad de Buenos Aires
6007010,Carhue
6007080,Rivera
6014010,Adolfo Gonzalez Chaves
6014020,De la Garma
6021010,Alberti
6021030,Mechita
6021070,Coronel Mom
6028010,Adrogué


In [14]:
#Se puede apreciar que a un codigo de localidad hay asociadas varias localidades, por lo tanto hay que crear claves unicas para cada localidad
biblio_c[biblio_c['cod_localidad'] == 6028010].groupby(['cod_localidad', 'localidad']).count().sort_values('cod_localidad')

id_provincia  id_departamento  \
cod_localidad localidad                                             
6028010       Adrogué                          4                4   
              Claypole                         1                1   
              Glew                             3                3   
              Libertad                         1                1   
              Longchamps                       2                2   
              San Francisco Solano             2                2   

                                    Observacion  categoria  Subcategoria  \
cod_localidad localidad                                                    
6028010       Adrogué                         4          4             4   
              Claypole                        1          1             1   
              Glew                            3          3             3   
              Libertad                        1          1             1   
              Longchamps                      2          2             2   
              San Francisco Solano            2          2             2   

                                    provincia  Departamento  nombre  \
cod_localidad localidad                                               
6028010       Adrogué                       4             4       4   
              Claypole                      1             1       1   
              Glew                          3             3       3   
              Libertad                      1             1       1   
              Longchamps                    2             2       2   
              San Francisco Solano          2             2       2   

                                    domicilio  Piso  ...  mail  web  \
cod_localidad localidad                              ...              
6028010       Adrogué                       4     4  ...     4    4   
              Claypole                      1     1  ...     1    1   
              Glew                          3     3  ...     3    3   
              Libertad                      1     1  ...     1    1   
              Longchamps                    2     2  ...     2    2   
              San Francisco Solano          2     2  ...     2    2   

                                    Información adicional  Latitud  Longitud  \
cod_localidad localidad                                                        
6028010       Adrogué                                   4        4         4   
              Claypole                                  1        1         1   
              Glew                                      3        3         3   
              Libertad                                  1        1         1   
              Longchamps                                2        2         2   
              San Francisco Solano                      2        2         2   

                                    TipoLatitudLongitud  fuente  Tipo_gestion  \
cod_localidad localidad                                                         
6028010       Adrogué                                 4       4             4   
              Claypole                                1       1             1   
              Glew                                    3       3             3   
              Libertad                                1       1             1   
              Longchamps                              2       2             2   
              San Francisco Solano                    2       2             2   

                                    año_inicio  Año_actualizacion  
cod_localidad localidad                                            
6028010       Adrogué                        4                  4  
              Claypole                       1                  1  
              Glew                           3                  3  
              Libertad                       1                  1  
              Longchamps              

In [15]:
biblio_c[['id_provincia','provincia']].groupby(['id_provincia','provincia']).count()

,
id_provincia,provincia
2,Ciudad Autónoma de Buenos Aires
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa


In [16]:
biblio_c[['id_departamento','Departamento']].groupby(['id_departamento','Departamento']).count().head(20)

,
id_departamento,Departamento
2000,Ciudad Autonoma de Buenos Aires
6007,Adolfo Alsina
6014,Adolfo Gonzales Chaves
6021,Alberti
6028,Almirante Brown
6035,Avellaneda
6042,Ayacucho
6049,Azul
6056,Bahia Blanca


Santa fe aparece escrito de 2 formas distintas

In [17]:
biblio_c['provincia'].replace('Santa Fé','Santa Fe', inplace=True)

El cod_localidad esta repetido, por lo que habra que corregirlo. Lo primero que se puede observar es que los primeros numeros del cod_localidad son los mismos que el id_departamento, asique voy a crear una clave hash que respete ese numero.

In [18]:
biblio_c['cod_localidad'] = biblio_c.apply(lambda row: mf.hash_function(row['id_departamento'],row['localidad']), axis=1)

In [19]:
biblio_c[['cod_localidad','localidad']].groupby(['cod_localidad','localidad']).count().head(20)

,
cod_localidad,localidad
100074965,Los Varelas
100218885,Andalgalá
100354326,Londres
100354450,Belén
100354868,Hualfin
100356329,Corral Quemado
100424050,Nueva Coneta
100424835,Chumbicha
100425290,Colonia del Valle


In [20]:
biblio_c[['id_provincia','provincia']].groupby(['id_provincia','provincia']).count()

,
id_provincia,provincia
2,Ciudad Autónoma de Buenos Aires
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa


In [21]:
biblio_c.isna().sum()

cod_localidad            0
id_provincia             0
id_departamento          0
Observacion              0
categoria                0
Subcategoria             0
provincia                0
Departamento             0
localidad                0
nombre                   0
domicilio                0
Piso                     0
codigo postal            0
Cod_tel                  0
numero de telefono       0
mail                     0
web                      0
Información adicional    0
Latitud                  0
Longitud                 0
TipoLatitudLongitud      0
fuente                   0
Tipo_gestion             0
año_inicio               0
Año_actualizacion        0
dtype: int64

<h1>NORMALIZANDO TABLA MUSEO</h1>

In [22]:
#museo_c.drop(columns=['Observaciones','piso','Info_adicional'], axis=1, inplace=True)

In [23]:
museo_c.rename(columns={'Cod_Loc': 'cod_localidad', 'IdProvincia': 'id_provincia','IdDepartamento': 'id_departamento','direccion': 'domicilio','CP': 'codigo postal','telefono': 'numero de telefono','Mail': 'mail','Web': 'web'}, inplace=True)

In [24]:
museo_c.fillna('s/d',inplace=True)

In [25]:
museo_c[['cod_localidad','localidad']].groupby(['cod_localidad','localidad']).count().head(20)

Empty DataFrame
Columns: []
Index: [(2000010, Ciudad Autónoma de Buenos Aires), (6007010, Carhue), (6028010, Burzaco), (6035010, Avellaneda), (6049020, Azul), (6056010, Bahía Blanca), (6056010, Villa Espora), (6063010, Balcarce), (6070010, Baradero), (6077010, Arrecifes), (6098010, Berisso), (6119050, Coronel Brandsen), (6189030, Coronel Dorrego), (6196010, Coronel Pringles), (6210020, Chacabuco), (6217010, Chascomus), (6224010, Chivilcoy), (6238010, Dolores), (6245010, Ensenada), (6260010, Monte Grande)]

In [26]:
museo_c['cod_localidad'] = museo_c.apply(lambda row: mf.hash_function(row['id_departamento'],row['localidad']), axis=1)

In [27]:
museo_c[['id_provincia','provincia']].groupby(['id_provincia','provincia']).count()

,
id_provincia,provincia
2,Ciudad Autónoma de Buenos Aires
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa


In [28]:
museo_c[(museo_c['id_provincia'] == 58) & (museo_c['provincia'] == 'Salta')]

,cod_localidad,id_provincia,id_departamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,domicilio,...,mail,web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
860,580353599,58,58035,s/d,Espacios de Exhibición Patrimonial,s/d,Salta,Salta,Museo Catedralicio,Av. Belgrano 589,...,s/d,s/d,-24.787762,-65.410492,Localización precisa,s/d,RCC,s/d,0.0,2017


In [29]:
museo_c.loc[860, 'id_provincia'] = 66

In [30]:
museo_c['provincia'].replace("Tierra del Fuego, Antártida e Islas del Atlántico Sur", 'Tierra del Fuego', inplace=True)

In [31]:
museo_c.isna().sum()

cod_localidad          0
id_provincia           0
id_departamento        0
Observaciones          0
categoria              0
subcategoria           0
provincia              0
localidad              0
nombre                 0
domicilio              0
piso                   0
codigo postal          0
cod_area               0
numero de telefono     0
mail                   0
web                    0
Latitud                0
Longitud               0
TipoLatitudLongitud    0
Info_adicional         0
fuente                 0
jurisdiccion           0
año_inauguracion       0
actualizacion          0
dtype: int64

<h1>NORMALIZANDO TABLA CINE</h1>

In [32]:
#cine_c.dropna(axis=1,thresh=cine_c.shape[0], inplace=True)

In [33]:
cine_c.columns

Index(['Cod_Loc', 'IdProvincia', 'IdDepartamento', 'Observaciones',
       'Categoría', 'Provincia', 'Departamento', 'Localidad', 'Nombre',
       'Dirección', 'Piso', 'CP', 'cod_area', 'Teléfono', 'Mail', 'Web',
       'Información adicional', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'tipo_gestion', 'Pantallas', 'Butacas', 'espacio_INCAA',
       'año_actualizacion'],
      dtype='object')

In [34]:
cine_c.rename(columns={'Cod_Loc': 'cod_localidad', 'IdProvincia': 'id_provincia','IdDepartamento': 'id_departamento','Categoría': 'categoria','Provincia': 'provincia','Localidad': 'localidad','Nombre': 'nombre','Dirección': 'domicilio','CP': 'codigo postal','Teléfono': 'numero de telefono','Mail': 'mail','Web': 'web', 'Fuente': 'fuente'}, inplace=True)

In [35]:
cine_c.fillna('s/d', inplace=True)

In [36]:
cine_c[['cod_localidad','localidad']].groupby(['cod_localidad','localidad']).count().head(30)

,
cod_localidad,localidad
2000010,Ciudad Autónoma de Buenos Aires
6014010,Gonzales Chaves
6028010,Adrogué
6035010,Avellaneda
6042010,Ayacucho
6049020,Azul
6056010,Bahía Blanca
6063010,Balcarce
6084020,Benito Juarez


In [37]:
cine_c['cod_localidad'] = cine_c.apply(lambda row: mf.hash_function(row['id_departamento'],row['localidad']), axis=1)

In [38]:
cine_c[['id_provincia','provincia']].groupby(['id_provincia','provincia']).count()

,
id_provincia,provincia
2,Ciudad Autónoma de Buenos Aires
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa


Normalizar toda la información de Museos, Salas de Cine y Bibliotecas Populares, para crear una única tabla que contenga:

o cod_localidad

o id_provincia

o id_departamento

o categoría

o provincia

o localidad

o nombre

o domicilio

o código postal

o número de teléfono

o mail

o web

In [39]:
mcb_norm = pd.concat([biblio_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web']], museo_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web']], cine_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web']]])

In [40]:
mcb_norm.reset_index(drop=True, inplace=True)

In [41]:
mcb_norm.head()

,cod_localidad,id_provincia,id_departamento,categoria,provincia,localidad,nombre,domicilio,codigo postal,numero de telefono,mail,web
0,700495604,70,70049,Bibliotecas Populares,San Juan,Rodeo,Biblioteca Popular Juan P. Garramuno,Santo Domingo,5465,s/d,s/d,s/d
1,20002551,2,2000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Helena Larroque de Roffo,Simbrón 3058,C1417EUD,45010078,asociacionroffo@yahoo.com.ar,s/d
2,20002551,2,2000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular 12 de Octubre,Calle Arengreen 1187,C1405CYM,49880766,popular12deoctubre@hotmail.com,s/d
3,20002551,2,2000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Villa Pueyrredón Norte,Cockrane 2334,C1419FMD,45729107,bibliotecapueyrredon@hotmail.com,s/d
4,20002551,2,2000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Alberdi,Acevedo 666,C1414DJN,47753325,bibliotecapopularalberdi@yahoo.com.ar,s/d


In [42]:
mcb_norm[['id_provincia','provincia']].groupby(['id_provincia','provincia']).count()

,
id_provincia,provincia
2,Ciudad Autónoma de Buenos Aires
6,Buenos Aires
10,Catamarca
14,Córdoba
18,Corrientes
22,Chaco
26,Chubut
30,Entre Ríos
34,Formosa


Hay un caracter extraño en la provincia de Neuquén

In [43]:
mcb_norm['provincia'] = mcb_norm.apply(lambda row: mf.arreglar_neuquen(row['provincia']), axis=1)

Procesar los datos conjuntos para poder generar una tabla con la siguiente información:

o Cantidad de registros totales por categoría

o Cantidad de registros totales por fuente

o Cantidad de registros por provincia y categoría

In [44]:
temp_table = pd.concat([biblio_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web', 'fuente']], museo_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web', 'fuente']], cine_c[['cod_localidad','id_provincia','id_departamento','categoria','provincia','localidad',
'nombre', 'domicilio','codigo postal','numero de telefono', 'mail', 'web', 'fuente']]])

In [45]:
total_cat = pd.DataFrame(temp_table.groupby('categoria').size(), columns=['cant_total'])
total_cat.reset_index(inplace=True)
total_cat

,categoria,cant_total
0,Bibliotecas Populares,2017
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,329


In [46]:
total_fuente = pd.DataFrame(temp_table.groupby('fuente').size(), columns=['cant_total'])
total_fuente.reset_index(inplace=True)
total_fuente

,fuente,cant_total
0,CNMLH - Enlace SInCA,1
1,CNMLH - Ente Cultural de Tucumán,1
2,CONABIP,2010
3,DNPyM,1048
4,Dirección Provincial de Patrimonio Cultural - ...,1
5,Dirección de Cultura de Villa Allende,2
6,Dirección de Cultura y de Educación de la Muni...,1
7,Gob. Pcia.,1
8,Gobierno de la Provincia,14
9,Gobierno de la Provincia de Chubut / SInCA 2013,3


In [47]:
total_prov_cat = pd.DataFrame(temp_table.groupby(['provincia', 'categoria']).size(), columns=['cant_total'])
total_prov_cat.reset_index(inplace=True)
total_prov_cat

,provincia,categoria,cant_total
0,Buenos Aires,Bibliotecas Populares,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,106
3,Catamarca,Bibliotecas Populares,38
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
67,Tierra del Fuego,Espacios de Exhibición Patrimonial,14
68,Tierra del Fuego,Salas de cine,3
69,Tucumán,Bibliotecas Populares,37
70,Tucumán,Espacios de Exhibición Patrimonial,26


Procesar la información de cines para poder crear una tabla que contenga:

o Provincia

o Cantidad de pantallas

o Cantidad de butacas

o Cantidad de espacios INCAA

In [48]:
cine_c['espacio_INCAA'].drop_duplicates()

0     s/d
1       0
11     si
56     SI
Name: espacio_INCAA, dtype: object

In [49]:
cine_c['espacio_INCAA'].replace('SI','si', inplace=True)

In [50]:
cine_c[cine_c['espacio_INCAA'] == '0']

,cod_localidad,id_provincia,id_departamento,Observaciones,categoria,provincia,Departamento,localidad,nombre,domicilio,...,Información adicional,Latitud,Longitud,TipoLatitudLongitud,fuente,tipo_gestion,Pantallas,Butacas,espacio_INCAA,año_actualizacion
1,100913306,10,10091,s/d,Salas de cine,Catamarca,Santa Maria,Santa María,Centro Cultural San Agustín,San Martin 173,...,s/d,-26.693859,-66.049058,Localización precisa,INCAA / SInCA,Privado comercial,1,440,0,2018


In [51]:
cine_c.groupby(['provincia','espacio_INCAA']).count().head(3)

cod_localidad  id_provincia  id_departamento  \
provincia    espacio_INCAA                                                 
Buenos Aires s/d                       86            86               86   
             si                        20            20               20   
Catamarca    0                          1             1                1   

                            Observaciones  categoria  Departamento  localidad  \
provincia    espacio_INCAA                                                      
Buenos Aires s/d                       86         86            86         86   
             si                        20         20            20         20   
Catamarca    0                          1          1             1          1   

                            nombre  domicilio  Piso  ...  web  \
provincia    espacio_INCAA                           ...        
Buenos Aires s/d                86         86    86  ...   86   
             si                 20         20    20  ...   20   
Catamarca    0                   1          1     1  ...    1   

                            Información adicional  Latitud  Longitud  \
provincia    espacio_INCAA                                             
Buenos Aires s/d                               86       86        86   
             si                                20       20        20   
Catamarca    0                                  1        1         1   

                            TipoLatitudLongitud  fuente  tipo_gestion  \
provincia    espacio_INCAA                                              
Buenos Aires s/d                             86      86            86   
             si                              20      20            20   
Catamarca    0                                1       1             1   

                            Pantallas  Butacas  año_actualizacion  
provincia    espacio_INCAA                                         
Buenos Aires s/d                   86       86                 86  
             si                    20       20                 20  
Catamarca    0                      1        1                  1  

[3 rows x 24 columns]

In [52]:
cine_c['espacio_INCAA'].replace('si',1, inplace=True)
cine_c['espacio_INCAA'].replace('0',0, inplace=True)
cine_c['espacio_INCAA'].replace('s/d',0, inplace=True)

In [53]:
cine_c['espacio_INCAA'].sum()

63

In [54]:
tabla_cine = mf.cine_redux(cine_c)
tabla_cine

,provincia,butacas,pantallas,espacio_INCAA
0,Catamarca,3200,12,1
1,Chaco,2469,14,1
2,Chubut,2682,10,4
3,Corrientes,3370,17,1
4,Entre Ríos,4086,17,2
5,Jujuy,2277,5,2
6,La Pampa,2071,6,2
7,La Rioja,1477,10,1
8,Misiones,2177,10,2
9,Salta,4665,23,2
